In [1]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession as ps
import sqlite3
import csv

The strategy for cleaning the sports.sqlite database will be to go table by table and assess what needs to be cleaned. 

First, we need to turn every sql table into a Pandas DF.

In [2]:
conn = sqlite3.connect('../archived data/sports.sqlite') 
spark = ps.builder.getOrCreate()
          
sql_query = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Player
                               ''', conn)
sql_query2 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Extra_Runs
                               ''', conn)
sql_query3 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Batting_Style
                               ''', conn)
sql_query4 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Bowling_style
                               ''', conn)
sql_query5 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Country
                               ''', conn)
sql_query6 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Season
                               ''', conn)
sql_query7 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM City
                               ''', conn)
sql_query8 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Outcome
                               ''', conn)
sql_query9 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Win_By
                               ''', conn)
sql_query10 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Wicket_Taken
                               ''', conn)
sql_query11 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Venue
                               ''', conn)
sql_query12 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Extra_Type
                               ''', conn)
sql_query13 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Out_Type
                               ''', conn)
sql_query14 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Toss_Decision
                               ''', conn)
sql_query15 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Umpire
                               ''', conn)
sql_query16 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Team
                               ''', conn)
sql_query17 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Ball_by_Ball
                               ''', conn)
sql_query18 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM sysdiagrams
                               ''', conn)
sql_query19 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM sqlite_sequence
                               ''', conn)
sql_query20 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Match
                               ''', conn)
sql_query21 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Rolee
                               ''', conn)
sql_query22 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Player_Match
                               ''', conn)
sql_query23 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Batsman_Scored
                               ''', conn)


player = pd.DataFrame(sql_query)
Extra_Runs = pd.DataFrame(sql_query2)
Batting_Style = pd.DataFrame(sql_query3)
Bowling_style = pd.DataFrame(sql_query4)
Country = pd.DataFrame(sql_query5)
Season = pd.DataFrame(sql_query6)
City = pd.DataFrame(sql_query7)
Outcome = pd.DataFrame(sql_query8)
Win_By = pd.DataFrame(sql_query9)
Wicket_Taken = pd.DataFrame(sql_query10)
Venue = pd.DataFrame(sql_query11)
Extra_Type = pd.DataFrame(sql_query12)
Out_Type = pd.DataFrame(sql_query13)
Toss_Decision = pd.DataFrame(sql_query14)
Umpire = pd.DataFrame(sql_query15)
Team = pd.DataFrame(sql_query16)
Ball_by_Ball = pd.DataFrame(sql_query17)
sysdiagrams = pd.DataFrame(sql_query18)
sqlite_sequence = pd.DataFrame(sql_query19)
Match = pd.DataFrame(sql_query20)
Roles = pd.DataFrame(sql_query21)
Player_Match = pd.DataFrame(sql_query22)
Batsman_Scored = pd.DataFrame(sql_query23)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/21 14:55:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/21 14:55:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


This database will be consolidated into 7 tables. run_plays, out_plays, umpires, matches, players, seasons, teams. These 6 are necessary to ensure that no data is lost in storage and the database can be extracted in the same way if necessary.

A rough draft of the Match table can be created by merging a number of tables together

In [3]:
match_season = pd.merge(Match, Season)
match_team = match_season.merge(Team, left_on=['Team_1'], right_on=['Team_Id'])
match_team = match_team.merge(Team, left_on=['Team_2'], right_on=['Team_Id'])
match_venue = match_team.merge(Venue)
match_venue = match_venue.merge(City)
match_toss = match_venue.merge(Toss_Decision, left_on=['Toss_Decide'], right_on=['Toss_Id'])
match_win = match_toss.merge(Win_By, left_on=['Win_Type'], right_on=['Win_Id'])
match_outcome = match_win.merge(Outcome, left_on=['Outcome_type'], right_on=['Outcome_Id'])
match_winner = match_outcome.merge(Team, left_on=['Match_Winner'], right_on=['Team_Id'])
match_man = match_winner.merge(player, left_on=['Man_of_the_Match'], right_on=['Player_Id'] )
match_twinner = match_man.merge(player, left_on=['Toss_Winner'], right_on=['Player_Id'] )
match_final = match_twinner.merge(Country, left_on=['Country_Name_y'], right_on=['Country_Id'])
match_final

,Match_Id,Team_1,Team_2,Match_Date,Season_Id,Venue_Id,Toss_Winner,Toss_Decide,Win_Type_x,Win_Margin,...,Bowling_skill_x,Country_Name_x,Player_Id_y,Player_Name_y,DOB_y,Batting_hand_y,Bowling_skill_y,Country_Name_y,Country_Id,Country_Name
0,335987,2,1,2008-04-18 00:00:00,1,1,2,1,1,140.0,...,1.0,4,2,BB McCullum,1981-09-27 00:00:00,2,1.0,4,4,New Zealand
1,548320,2,1,2012-04-10 00:00:00,5,1,2,1,1,42.0,...,5.0,1,2,BB McCullum,1981-09-27 00:00:00,2,1.0,4,4,New Zealand
2,501260,2,1,2011-05-14 00:00:00,4,1,2,1,2,4.0,...,2.0,8,2,BB McCullum,1981-09-27 00:00:00,2,1.0,4,4,New Zealand
3,598013,2,1,2013-04-11 00:00:00,6,1,2,1,2,8.0,...,2.0,8,2,BB McCullum,1981-09-27 00:00:00,2,1.0,4,4,New Zealand
4,501271,2,3,2011-05-22 00:00:00,4,1,2,1,2,8.0,...,2.0,8,2,BB McCullum,1981-09-27 00:00:00,2,1.0,4,4,New Zealand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,336041,5,7,2008-05-26 00:00:00,1,6,5,1,2,5.0,...,9.0,6,5,Mohammad Hafeez,1980-10-17 00:00:00,2,2.0,6,6,Pakistan
570,501236,5,7,2011-04-29 00:00:00,4,6,5,1,2,7.0,...,2.0,2,5,Mohammad Hafeez,1980-10-17 00:00:00,2,2.0,6,6,Pakistan
571,733994,5,11,2014-05-08 00:00:00,7,19,5,1,1,32.0,...,1.0,1,5,Mohammad Hafeez,1980-10-17 00:00:00,2,2.0,6,6,Pakistan
572,829792,5,11,2015-05-07 00:00:00,8,18,5,1,1,7.0,...,1.0,10,5,Mohammad Hafeez,1980-10-17 00:00:00,2,2.0,6,6,Pakistan


With the merging done now we just need to clean the table

In [4]:

match_final = match_final.drop(['Team_1', 'Team_2', 'Season_Id', 'Venue_Id', 'Toss_Winner', 'Win_Type_x', 'Man_of_the_Match', 'Man_of_the_Series', 'Outcome_type', 'Orange_Cap', 'Purple_Cap', 'Team_Id_x', 'Country_Name_x', 'Team_Id_y', 'Player_Id_y', 'Country_Id', 'Country_Name_y', 'Bowling_skill_y', 'Batting_hand_y', 'DOB_y', 'Player_Id_y', 'DOB_x', 'Batting_hand_x', 'Bowling_skill_x', 'Team_Id', 'Player_Id_x', 'City_Id', 'Toss_Id', 'Win_Id', 'Outcome_Id', 'Country_id', 'Match_Winner'], axis=1)
match_final = match_final.rename(columns={'Team_Name_x' : 'Team_1', 'Team_Name_y' : 'Team_2', 'Win_Type_y' : 'Win_Type', 'Player_Name_x' : 'Toss_Winner', 'Player_Name_y' : 'Man_of_the_Match', 'Team_Name' : 'Match_Winner'})
match_final = match_final.reset_index(names='id')
match_final

,id,Match_Id,Match_Date,Toss_Decide,Win_Margin,Season_Year,Team_1,Team_2,Venue_Name,City_Name,Toss_Name,Win_Type,Outcome_Type,Match_Winner,Toss_Winner,Man_of_the_Match,Country_Name
0,0,335987,2008-04-18 00:00:00,1,140.0,2008,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,Bangalore,field,runs,Result,Kolkata Knight Riders,BB McCullum,BB McCullum,New Zealand
1,1,548320,2012-04-10 00:00:00,1,42.0,2012,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,Bangalore,field,runs,Result,Kolkata Knight Riders,L Balaji,BB McCullum,New Zealand
2,2,501260,2011-05-14 00:00:00,1,4.0,2011,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,Bangalore,field,wickets,Result,Royal Challengers Bangalore,CH Gayle,BB McCullum,New Zealand
3,3,598013,2013-04-11 00:00:00,1,8.0,2013,Royal Challengers Bangalore,Kolkata Knight Riders,M Chinnaswamy Stadium,Bangalore,field,wickets,Result,Royal Challengers Bangalore,CH Gayle,BB McCullum,New Zealand
4,4,501271,2011-05-22 00:00:00,1,8.0,2011,Royal Challengers Bangalore,Chennai Super Kings,M Chinnaswamy Stadium,Bangalore,field,wickets,Result,Royal Challengers Bangalore,CH Gayle,BB McCullum,New Zealand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,569,336041,2008-05-26 00:00:00,1,5.0,2008,Rajasthan Royals,Mumbai Indians,Sawai Mansingh Stadium,Jaipur,field,wickets,Result,Rajasthan Royals,Sohail Tanvir,Mohammad Hafeez,Pakistan
570,570,501236,2011-04-29 00:00:00,1,7.0,2011,Rajasthan Royals,Mumbai Indians,Sawai Mansingh Stadium,Jaipur,field,wickets,Result,Rajasthan Royals,J Botha,Mohammad Hafeez,Pakistan
571,571,733994,2014-05-08 00:00:00,1,32.0,2014,Rajasthan Royals,Sunrisers Hyderabad,"Sardar Patel Stadium, Motera",Ahmedabad,field,runs,Result,Sunrisers Hyderabad,B Kumar,Mohammad Hafeez,Pakistan
572,572,829792,2015-05-07 00:00:00,1,7.0,2015,Rajasthan Royals,Sunrisers Hyderabad,Brabourne Stadium,Mumbai,field,runs,Result,Sunrisers Hyderabad,EJG Morgan,Mohammad Hafeez,Pakistan


Moving on to Run_Plays - Same process

In [5]:
ball_rough = Ball_by_Ball.merge(Extra_Runs)
ball_scores = ball_rough.merge(Batsman_Scored, left_on=['Match_Id'], right_on=['Match_Id'])
ball_scores = ball_scores.drop(['Over_Id_x', 'Ball_Id_x', 'Innings_No_x', 'Striker_Batting_Position'], axis=1)
ball_team = ball_scores.merge(Team, left_on=['Team_Batting'], right_on=['Team_Id'])
ball_team = ball_team.merge(Team, left_on=['Team_Bowling'], right_on=['Team_Id'])
ball_team = ball_team.merge(Extra_Type, left_on=['Extra_Type_Id'], right_on=['Extra_Id'])
ball_team = ball_team.drop(['Extra_Type_Id', 'Team_Batting', 'Team_Bowling', 'Team_Id_x', 'Team_Id_y', 'Extra_Id'], axis=1)
ball_runs_final = ball_team.rename(columns={'Over_Id_y' : 'Over_Id', 'Ball_Id_y' : 'Ball_Id', 'Innings_No_y' : 'Innings_No', 'Team_Name_x' : 'Team_Batting', 'Team_Name_y' : 'Team_Bowling'})
ball_runs_final

,Match_Id,Striker,Non_Striker,Bowler,Extra_Runs,Over_Id,Ball_Id,Runs_Scored,Innings_No,Team_Batting,Team_Bowling,Extra_Name
0,335987,1,2,14,1,1,1,0,1,Kolkata Knight Riders,Royal Challengers Bangalore,legbyes
1,335987,1,2,14,1,1,1,1,2,Kolkata Knight Riders,Royal Challengers Bangalore,legbyes
2,335987,1,2,14,1,1,2,0,1,Kolkata Knight Riders,Royal Challengers Bangalore,legbyes
3,335987,1,2,14,1,1,3,0,2,Kolkata Knight Riders,Royal Challengers Bangalore,legbyes
4,335987,1,2,14,1,1,4,0,1,Kolkata Knight Riders,Royal Challengers Bangalore,legbyes
...,...,...,...,...,...,...,...,...,...,...,...,...
1734268,335993,41,42,131,5,20,3,1,1,Delhi Daredevils,Deccan Chargers,penalty
1734269,335993,41,42,131,5,20,4,1,1,Delhi Daredevils,Deccan Chargers,penalty
1734270,335993,41,42,131,5,20,5,2,1,Delhi Daredevils,Deccan Chargers,penalty
1734271,335993,41,42,131,5,20,6,6,1,Delhi Daredevils,Deccan Chargers,penalty


In [6]:
ball_rough2 = Ball_by_Ball.merge(Wicket_Taken)
ball_team2 = ball_rough2.merge(Team, left_on=['Team_Batting'], right_on=['Team_Id'])
ball_team2 = ball_team2.merge(Team, left_on=['Team_Bowling'], right_on=['Team_Id'])
ball_player = ball_team2.merge(player, left_on=['Player_Out'], right_on=['Player_Id'])
ball_player = ball_player.merge(Out_Type, left_on=['Kind_Out'], right_on=['Out_Id'])
ball_player = ball_player.drop(['Team_Batting', 'Team_Bowling', 'Bowler', 'Team_Id_y', 'Player_Id', 'Batting_hand', 'Bowling_skill', 'Country_Name', 'DOB', 'Team_Id_x', 'Kind_Out', 'Fielders', 'Player_Out', 'Striker_Batting_Position', 'Striker', 'Non_Striker'], axis=1)
ball_player = ball_player.rename(columns={'Team_Name_x' : 'Team_Batting', 'Team_Name_y' : 'Team_Bowling', 'Player_Name' : 'Player_Out', 'Out_Name' : 'Out_Type', 'Over_Id_y' : 'Over_Id', 'Ball_Id_y' : 'Ball_Id', 'Innings_No_y' : 'Innings_No'})
ball_player

,Match_Id,Over_Id,Ball_Id,Innings_No,Team_Batting,Team_Bowling,Player_Out,Out_Id,Out_Type
0,335987,2,1,2,Royal Challengers Bangalore,Kolkata Knight Riders,R Dravid,2,bowled
1,336015,10,1,2,Royal Challengers Bangalore,Kolkata Knight Riders,R Dravid,2,bowled
2,419114,13,1,1,Royal Challengers Bangalore,Kolkata Knight Riders,R Dravid,2,bowled
3,501214,12,3,1,Rajasthan Royals,Kolkata Knight Riders,R Dravid,2,bowled
4,598010,8,5,1,Rajasthan Royals,Kolkata Knight Riders,R Dravid,2,bowled
...,...,...,...,...,...,...,...,...,...
6722,392207,6,3,2,Rajasthan Royals,Chennai Super Kings,SA Asnodkar,8,hit wicket
6723,980988,18,6,1,Sunrisers Hyderabad,Delhi Daredevils,DJ Hooda,8,hit wicket
6724,336021,5,6,1,Royal Challengers Bangalore,Kings XI Punjab,Misbah-ul-Haq,8,hit wicket
6725,335996,18,2,2,Mumbai Indians,Kings XI Punjab,MA Khote,8,hit wicket


Player should be simpler:

In [7]:
player_country = player.merge(Country, left_on=['Country_Name'], right_on=['Country_Id'])
player_country = player_country.drop(['Country_Name_x', 'Country_Id'], axis=1)
player_country = player_country.rename(columns={'Country_Name_y' : 'Country_Name'})
player_country

,Player_Id,Player_Name,DOB,Batting_hand,Bowling_skill,Country_Name
0,1,SC Ganguly,1972-07-08 00:00:00,1,1.0,India
1,6,R Dravid,1973-01-11 00:00:00,2,2.0,India
2,7,W Jaffer,1978-02-16 00:00:00,2,2.0,India
3,8,V Kohli,1988-11-05 00:00:00,2,1.0,India
4,12,B Akhil,1977-10-07 00:00:00,2,5.0,India
...,...,...,...,...,...,...
464,195,Mashrafe Mortaza,1983-10-05 00:00:00,2,3.0,Bangladesh
465,202,Mohammad Ashraful,1984-07-07 00:00:00,2,2.0,Bangladesh
466,276,Shakib Al Hasan,1987-03-24 00:00:00,1,7.0,Bangladesh
467,460,Mustafizur Rahman,1995-09-06 00:00:00,1,6.0,Bangladesh


Finally let's merge all of the venue information into 1 table

In [8]:
venue_city = Venue.merge(City)
venue_country = venue_city.merge(Country, left_on=['Country_id'], right_on=['Country_Id'])
venue_country = venue_country.drop(['Country_id'], axis=1)
venue_country

,Venue_Id,Venue_Name,City_Id,City_Name,Country_Id,Country_Name
0,1,M Chinnaswamy Stadium,1,Bangalore,1,India
1,2,"Punjab Cricket Association Stadium, Mohali",2,Chandigarh,1,India
2,33,"Punjab Cricket Association IS Bindra Stadium, ...",2,Chandigarh,1,India
3,3,Feroz Shah Kotla,3,Delhi,1,India
4,4,Wankhede Stadium,4,Mumbai,1,India
5,9,Dr DY Patil Sports Academy,4,Mumbai,1,India
6,18,Brabourne Stadium,4,Mumbai,1,India
7,5,Eden Gardens,5,Kolkata,1,India
8,6,Sawai Mansingh Stadium,6,Jaipur,1,India
9,7,"Rajiv Gandhi International Stadium, Uppal",7,Hyderabad,1,India


We now have every table we need to export into the Data-Warehouse. Pandas has a built in to_sql() function that can make this easier.

In [9]:
db = sqlite3.connect('../data-warehouse.db')
match_final.to_sql('matches', con=db, if_exists='append', schema='matches', index=False)

db.commit()
db.close()

In [10]:
db = sqlite3.connect('../data-warehouse.db')

ball_runs_final.to_sql('ball_runs', con=db, if_exists='append', schema='ball_runs', index=False)

db.commit()
db.close()

In [11]:
db = sqlite3.connect('../data-warehouse.db')

ball_player.to_sql('ball_outs', con=db, if_exists='append', schema='ball_outs', index=False)

db.commit()
db.close()

In [12]:
db = sqlite3.connect('../data-warehouse.db')

player_country.to_sql('players', con=db, if_exists='append', schema='players', index=False)

db.commit()
db.close()

In [13]:
db = sqlite3.connect('../data-warehouse.db')

venue_country.to_sql('venues', con=db, if_exists='append', schema='venues', index=False)

db.commit()
db.close()

In [14]:
db = sqlite3.connect('../data-warehouse.db')

Season.to_sql('seasons', con=db, if_exists='append', schema=None, index=False)
Umpire.to_sql('umpires', con=db, if_exists='append', schema=None, index=False)
Team.to_sql('teams', con=db, if_exists='append', schema=None, index=False)
Roles.to_sql('roles', con=db, if_exists='append', schema=None, index=False)

db.commit()
db.close()

23/05/21 14:56:06 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
